In [10]:
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install ast
%pip install logging
%pip install seaborn
%pip install argparse
%pip install statsmodels


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/d8/yhrnl4gn78vbcvy365vfz_q00000gp/T/pip-install-acy2p5yv/ast_ca6d3f3266854e08aca38d6069ce001e/setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "<frozen codecs>", line 918, in open
      FileNotFoundError: [Errno 2] No such file or di

In [9]:
import pandas as pd
import ast
import statsmodels.api as sm
import argparse
import logging

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

### **Analysis of Variance Linear Model**


ANOVA_LM (Analysis of Variance Linear Model) is a statistical method used in regression analysis to:

Decompose variance in a dependent variable

Assess the significance of predictors in a linear regression model

Test whether different groups or factors have statistically significant effects on the outcome

-   **Most Important for Hypothesis Testing**: **p-value** (PR(>F)) --- it directly determines if you can reject the null hypothesis.
    -   **If p-value < 0.05**: There is sufficient evidence to **reject the null hypothesis**, suggesting that at least one group mean differs from the others (i.e., the factor has a significant effect).
    -   **If p-value > 0.05**: There is insufficient evidence to reject the null hypothesis, meaning that **there is no significant difference** between the group means, and the factor does not have a significant effect.

-   **Supporting Information**: F-statistic helps you understand the size of the effect, and the sum of squares and degrees of freedom help to compute the F-statistic but are not as directly relevant for hypothesis testing.

In [ ]:
# This is as two way ANOVA ( only comparing TWO variables )
data.columns = data.columns.str.replace(' ', '_')

# 'Babble_Length' need to be the last colum in column_names

column_names = data[["Sex", "Age_on_bout_day", "Babble_Length"]]  
# column_names = data[["Age_on_bout_day", "Nestling_ID", "Babble_Length"]]  
# column_names = data[["Age_on_bout_day", "Fledge_age", "Babble_Length"]]  

print("Columns selected:", column_names.columns.tolist())

factors = column_names.columns[:-1]
response = column_names.columns[-1]

# Construct the formula for the ANOVA (using interaction if needed)
formula = f'{response} ~ ' + ' * '.join(factors)
# formula = f'{response} ~ ' + (factors[0])

print(formula) 

model = ols(formula, data=column_names).fit()
anova_result = anova_lm(model)
print(anova_result)


# What Comparisons Test I Have Ran

# Babble_Length ~ Age on bout day * Nestling ID
# Babble_Length ~ Sex * Treatment




# Babble_Length ~ Sex * Age
# Babble_Length ~ Age * Treatment
# Babble_Length ~ Sex * Age * Treatment


In [ ]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import argparse
import logging
import ast

# Initialize logging
logging.basicConfig(level=logging.INFO)

# Function to preprocess the data
def preprocess_data(data):
    logging.info('Preprocessing data...')

    # Map 'Sex' and 'Treatment' columns to numeric values
    data['Sex'] = data['Sex'].map({'M': 0, 'F': 1})
    data['Treatment'] = data['Treatment'].map({'CONTROL': 0, 'CORT': 1, 'OIL': 2})

    # Convert strings in 'Babbles' column to lists
    data['Babbles'] = data['Babbles'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Convert date columns to datetime
    date_columns = ['Hatch date', 'Fledge date', 'Date on vocalization']
    for col in date_columns:
        data[col] = pd.to_datetime(data[col], errors='coerce')

    # Extract statistics from 'Babbles' column
    def process_babbles(babble_list):
        return {
            'babble_count': len(babble_list),
            'babble_mean': sum(babble_list) / len(babble_list) if babble_list else 0,
            'babble_sum': sum(babble_list),
        }

    babbles_stats = data['Babbles'].apply(process_babbles)
    data['Babble Length'] = babbles_stats.apply(lambda x: x['babble_count'])
    data['Babble Mean'] = babbles_stats.apply(lambda x: x['babble_mean'])
    data['Babble Sum'] = babbles_stats.apply(lambda x: x['babble_sum'])

    logging.info('Data preprocessing completed.')
    return data

# Function to compute ANOVA
def compute_anova(data, factors, response):
    # Ensure the response is the last column in the formula
    formula = f"{response} ~ " + " * ".join(factors)
    logging.info(f"ANOVA Formula: {formula}")
    
    # Fit the model
    model = ols(formula, data=data).fit()
    
    # Compute ANOVA
    anova_result = anova_lm(model)
    logging.info("\nANOVA Results:")
    logging.info(anova_result)
    
    # Check if p-value is significant
    significant = anova_result["PR(>F)"].min() < 0.05
    logging.info(f"\nSignificant P-Value Found: {'Yes' if significant else 'No'}")
    return anova_result

def main():
    # Parse command-line arguments
    parser = argparse.ArgumentParser(description="Perform ANOVA on user-selected columns.")
    parser.add_argument("-i", "--input",   type=str, default="../CMBabble_Master_clean.csv", help="Path to the CSV file")
    parser.add_argument("-f", "--factors", type=lambda s: [item.strip() for item in s.split(',')], default="Sex, Age_on_bout_day", help="List of independent variables separated by commas.")
    # parser.add_argument("-f", "--factors", default="Sex", help="List of independent variables separated by commas.")
    args = parser.parse_args()

    # Hardcoded dependent variable (response) - Ensure column names are standardized Anova can read spaces, replace with '_'
    response = "Babble_Length"

    # Load the dataset or use the default
    logging.info('Reading and preparing data for analysis')
    if args.input:
        data = pd.read_csv(args.input)
    else:
        logging.warning("No data file provided, default dataset will be used.")

    # Preprocess the data
    data = preprocess_data(data)

    selected_columns = data[["Sex", "Babble Length"]]  
    
    # print("Columns selected:", column_names.columns.tolist())

    try:
        # selected_data = data[selected_columns]
        selected_data = data[["Sex", "Babble Length"]]  

    except KeyError as e:
        print(f"Error: {e}")
        print("Ensure the selected columns exist in the DataFrame or CSV file.")
        return

    print(f"Columns selected for ANOVA: {selected_columns}")

    # Ensure column names are standardized
    data.columns = data.columns.str.replace(' ', '_')
    
    # Compute ANOVA
    compute_anova(selected_data, args.factors, response)

if __name__ == "__main__":
    main()


# Variables to Examine or Test from ANOVA Testing Table

### Single Variables 

- **Age on bout day:** The p-value is (1.75e-17), so reject the null hypothesis. This indicates that Age on bout day significantly affects the dependent variable.

- **Nestling ID:** The p-value is (1.463254e-04), so reject the null hypothesis. This indicates that Nestling ID significantly affects the dependent variable.

### Two Variables 


___

# Variables to Exclude

- **Age on bout day:Nestling_ID:** The p-value is 0.341, which is greater than 0.05, so fail to reject the null hypothesis. This indicates that there is no significant interaction between Age_on_bout_day and Nestling_ID on the dependent variable.

- **Sex:Treatment:** The p-value is 0.120709, which is greater than 0.05, so fail to reject the null hypothesis. This indicates that there is no significant interaction between Sex and Treatment on the dependent variable.



